In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from cassandra.cluster import Cluster

In [2]:
spark = SparkSession.builder \
    .appName("ElhubSilver") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/10/13 13:22:38 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.22 instead (on interface en0)
25/10/13 13:22:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c940c31f-9a7f-4e4f-b062-2bbfbd90ab75;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found org.apache.cassandra#java-driver-mapper-runtime;4.18.1 in central
	found org.apache.cassandra#java-driver-query-builder;4.18.1 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.19 in central
:: resolution report :: resolve 240ms :: artifacts dl 

✅ SparkSession started with Cassandra integration


In [3]:
bronze_df = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_raw", keyspace="elhub_data")
    .load()
)

In [4]:
silver_df = (
    bronze_df
    .select(
        col("priceArea").alias("pricearea"),
        col("productionGroup").alias("productiongroup"),
        col("startTime").alias("starttime"),
        col("quantityKwh").alias("quantitykwh")
    )
    .withColumn("starttime", col("starttime").cast("timestamp"))
    .withColumn("quantitykwh", col("quantitykwh").cast("double"))
)

In [ ]:
cluster = Cluster(["localhost"])
session = cluster.connect("elhub_data")
session.execute("TRUNCATE production_silver")

In [ ]:
(
    silver_df
    .write
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .mode("append")  
    .save()
)
print("✅ Data written to elhub_data.production_silver (after truncate)")

✅ Data skrevet til elhub_data.production_silver (etter truncate)


In [ ]:
# Check for missing values
missing_values = silver_df.filter(
    col("pricearea").isNull() |
    col("productiongroup").isNull() |
    col("starttime").isNull() |
    col("quantitykwh").isNull()
).count()
if missing_values == 0:
    print("✅ No missing values in silver_df")
else:
    print(f"⚠️ There are {missing_values} rows with missing values in silver_df")

✅ Ingen manglende verdier i silver_df


In [18]:
# Describe the dataframe
silver_df.describe().show()

+-------+---------+---------------+------------------+
|summary|pricearea|productiongroup|       quantitykwh|
+-------+---------+---------------+------------------+
|  count|   215353|         215353|            215353|
|   mean|     NULL|           NULL| 729647.3826658479|
| stddev|     NULL|           NULL|1549543.7955768798|
|    min|      NO1|          hydro|               0.0|
|    max|      NO5|           wind|         9715193.0|
+-------+---------+---------------+------------------+

